In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install ktrain

     |████████████████████████████████| 25.3 MB 101 kB/s 
     |████████████████████████████████| 6.8 MB 15.3 MB/s 
     |████████████████████████████████| 981 kB 49.6 MB/s 
     |████████████████████████████████| 263 kB 36.7 MB/s 
     |████████████████████████████████| 1.9 MB 35.8 MB/s 
     |████████████████████████████████| 1.2 MB 36.6 MB/s 
     |████████████████████████████████| 468 kB 46.3 MB/s 
     |████████████████████████████████| 895 kB 13.2 MB/s 
     |████████████████████████████████| 3.3 MB 37.3 MB/s 
  Created wheel for ktrain: filename=ktrain-0.27.3-py3-none-any.whl size=25283131 sha256=88005833d1dba3d0c778a84e610adbc0bc4947f943b99c5aa8919079b020f504
  Stored in directory: /root/.cache/pip/wheels/3c/ef/e1/7da805d8a5944e8a3ac0553831d832c00b1800b848939849cb
  Created wheel for seqeval: filename=seqeval-0.0.19-py3-none-any.whl size=9929 sha256=b5b49d638c6639cd5e4b790b65bdff87b106f8041fc92138c42acd3d7feb0412
  Stored in directory: /root/.cache/pip/wheels/f5/ac/f1/4e13d7aff

In [3]:
import pandas as pd
import numpy as np
import ktrain
from ktrain import text

In [4]:
Movie_review_df = pd.read_csv("/content/drive/MyDrive/NLP/MovieReviewNLP/IMDB-Dataset.csv", nrows = 5000)
Movie_review_df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
train_df = Movie_review_df.loc[0:4001]
test_df = Movie_review_df.loc[4001:]

In [6]:
(X_train, y_train), (X_test, y_test), preprocess = text.texts_from_df(train_df=train_df,
                                                                      text_column = 'review',
                                                                      label_columns = 'sentiment',
                                                                      val_df = test_df,
                                                                      maxlen = 512,
                                                                      preprocess_mode = 'bert'
                                                                      )

['negative', 'positive']
   negative  positive
0       0.0       1.0
1       0.0       1.0
2       0.0       1.0
3       1.0       0.0
4       0.0       1.0
['negative', 'positive']
      negative  positive
4001       0.0       1.0
4002       0.0       1.0
4003       0.0       1.0
4004       1.0       0.0
4005       0.0       1.0
downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [7]:
model = text.text_classifier(name='bert',
                             train_data = (X_train, y_train),
                             preproc = preprocess
                             )

Is Multi-Label? False
maxlen is 512
done.


In [8]:
# get Learning rate
learner = ktrain.get_learner(model=model,
                             train_data = (X_train, y_train),
                             val_data = (X_test, y_test),
                             batch_size = 5
                             )

In [9]:
###To find the best suitable learning rate to use
#learner.lr_find()
#learner.lr_plot()

In [10]:
learner.fit_onecycle(lr=1e-5, epochs=1)



begin training using onecycle policy with max lr of 1e-05...
801/801 [==============================] - 1159s 1s/step - loss: 0.3831 - accuracy: 0.8156 - val_loss: 0.1695 - val_accuracy: 0.9409


In [11]:
predictor = ktrain.get_predictor(learner.model, preprocess)

In [12]:
data = ["It was a very good movie. will definitely recomment it to my friends and family",
        "The most hatted movie ever in the history of mankind, it is the worst movie"]

In [13]:
predictor.predict(data)

/usr/local/lib/python3.7/dist-packages/ktrain/text/preprocessor.py:215: UserWarning: List or array of two texts supplied, so task being treated as text classification. If this is a sentence pair classification task, please cast to tuple.
  warnings.warn('List or array of two texts supplied, so task being treated as text classification. ' +\


['positive', 'negative']

In [14]:
# save model
model.save('/content/Movie_review_model_BERT')

/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
